In [1]:
import sys
from matplotlib import rcParams 
rcParams["figure.figsize"] = (16,5)
sys.path.append('')

import librosa
import librosa.display

from TTS.tts.models.tacotron2 import Tacotron2 
from TTS.tts.layers import *
from TTS.tts.utils.data import *
from TTS.utils.audio import AudioProcessor
from TTS.utils.io import load_config
from TTS.tts.utils.text import text_to_sequence, phoneme_to_sequence
from TTS.tts.utils.text.symbols import symbols, phonemes
from TTS.utils.generic_utils import (KeepAverage, count_parameters,
                                     create_experiment_folder, get_git_branch,
                                     remove_experiment_folder, set_init_dict)

import IPython
from IPython.display import Audio
from TTS.tts.utils import *
from TTS.tts.utils.generic_utils import setup_model

from TTS.tts.utils.visual import visualize, plot_alignment, plot_spectrogram
import matplotlib.pyplot as plt
%matplotlib inline
import torch

In [2]:
# Set constants

# MODEL_PATH = '/content/drive/MyDrive/Mestrado/TTS/GST_rosana_only/checkpoint_120000.pth.tar'
MODEL_PATH = './experiments/debug_linear_logits/best_model.pth.tar'
CONFIG_PATH =  './experiments/debug_linear_logits/config.json'

# MODEL_PATH ='/content/drive/MyDrive/Mestrado/TTS/GST_3speaker_CPQD/best_model.pth.tar'
# CONFIG_PATH = '/content/drive/MyDrive/Mestrado/TTS/GST_3speaker_CPQD/config.json'

CONFIG = load_config(CONFIG_PATH)
# CONFIG['datasets'][0]['path'] = './'
# CONFIG['output_path'] = './'
# CONFIG['audio']['signal_norm'] = False
# CONFIG['audio']['stats_path'] = ''
# CONFIG['use_phonemes'] = False
# CONFIG['save_step'] = 500
use_cuda = True

# load the model
ap = AudioProcessor(**CONFIG.audio)

num_chars = len(phonemes) if CONFIG.use_phonemes else len(symbols)


# load the model
num_chars = len(phonemes) if CONFIG.use_phonemes else len(symbols)
model = setup_model(num_chars, 3, 2, CONFIG, None)
# cp = torch.load(MODEL_PATH, map_location=torch.device('cpu'))
# cp = torch.load(MODEL_PATH, map_location=torch.device('cpu'))
# model.load_state_dict(cp['model'])
model.eval()

if use_cuda:
    model.cuda()

# model.decoder.set_r(cp['r'])

 > Setting up Audio Processor...
 | > sample_rate:16000
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:50.0
 | > mel_fmax:7600.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > stats_path:
 | > hop_length:256
 | > win_length:1024
hi
 > Using model: Tacotron2


In [3]:
text_cleaner = [CONFIG.text_cleaner]
text_cleaner

['phoneme_cleaners']

In [4]:
# s = '''Um professor, para se tornar doutor, investe, pelo menos, 8 anos de sua vida ( se partir da graduação direto pro doutorado). Se fizer graduação, mestrado e doutorado, são 10 anos. Se fizer graduação, especialização, mestrado e doutorado, lá se vão 12 anos. Sim, 12 anos de universidade. Todo esse tempo labutando com ensino, pesquisa e extensão. Produzindo, apresentando e publicando trabalhos.  Participando de debates e congressos. Dando aula. Ministrando cursos. Participando de bancas. Emitindo pareceres. Orientando artigos, projetos de conclusão de cursos, TCC's e etc, etc, etc. De repente, todo esse conhecimento arduamente produzido dentro dos parâmetros científicos e acadêmicos passa a não valer  nada ante a palavra de um esquizóide qualquer que define a forma como nós, professores, devemos conduzir o trabalho para o qual nos preparamos ao longo da vida e diuturnamente - pesquisando e estudando métodos e teorias, produzindo e submetendo nossas ideias à comunidade científica - que, todos sabemos, não é muito conhecida pela generosidade. Sem comentar os diversos perrengues -  familiares, socioemocionais, financeiros e de saúde -  que enfrentamos durante e  em virtude dessa caminhada extenuante e tão mal recompensada. Ainda assim, contra tudo e todos, persistimos cantando no mal tempo, porque temos a certeza daquilo que fazemos, e assentamos nossa autoridade e dignidade no conhecimento que produzimos. 
# Antes, a violência que se abatia sobre nós, profissionais da educação, era apenas financeira e a desvalorização social era decorrente dessa desvalorização financeira: '' Você é só uma professorazinha, ganha pouco''. 
# Agora, a violência é pior: a palavra e a autoridade intelectual do professor -  que ainda mantínhamos, apesar de tudo -  são jogados por terra. 
# Somos confrontados por pessoas que não leem, não têm conhecimento de absolutamente nada e que se informam apenas por youtubers, líderes religiosos delirantes ou figuras obscuras sem nenhum (re)conhecimento científico e que são  merecidamente proscritas do meio acadêmico.  Ainda por cima, somos chamados de '' doutrinadores'', acusados e vilipendiados das piores formas possíveis sem que os difamadores se retratem ou sofram as devidas punições por suas afirmações criminosas que têm como intuito deslegitimar, mais uma vez, a profissão docente nos violentando moralmente. E, como se não bastasse, há ''professores'' que se alinham a esse  evidente projeto de destruição da educação no país. Não me admira que a profissão esteja entre as menos procuradas pelos jovens, e que esteja, infelizmente, caminhando para a extinção.'''

# s = "ambulantes vendem os ingressos para a copa do mundo de dois mil e catorze , mas só alguns dias antes do dia do sorteio ."

# s = "baixando o aplicativo do trivago , você vai encontrar o hotel dos sonhos e com preços ótimos ."

s = "não pode haver tréguas na guerra contra a malária ."

# s = "e é"

seq = np.asarray(text_to_sequence(s, text_cleaner), dtype=np.int32)
chars_var = torch.from_numpy(seq).unsqueeze(0).cuda()

seq

array([ 60,  47,  61, 101,  62,  61,  50,  51, 101,  54,  47,  68,  51,
        64, 101,  66,  64,  51,  53,  67,  47,  65, 101,  60,  47, 101,
        53,  67,  51,  64,  64,  47, 101,  49,  61,  60,  66,  64,  47,
       101,  47, 101,  59,  47,  58,  47,  64,  55,  47, 101,  97])

In [5]:
centroid_animado = np.array([[-4.76433074e-02, -6.83329008e-01, -5.10745978e-01,
        -1.57049902e-02,  5.02526537e-02, -5.58714376e-03,
        -1.88081062e-01,  4.56058863e-01,  2.26621968e-02,
        -5.31872478e-01, -3.13436115e-01, -8.41244559e-01,
        -6.05470923e-03, -1.37813421e-01, -1.63020981e-01,
        -3.75179609e-02,  7.05922417e-02,  3.98711837e-02,
         1.42862546e-01, -3.82136639e-01,  1.48595868e-02,
        -4.04462285e-02, -1.73014231e-01,  4.58017268e-02,
        -7.41732786e-02,  8.45176698e-02,  2.11554609e-01,
         7.42359138e-04, -8.76026817e-03, -3.81261229e-01,
        -1.34325071e-01, -1.76209638e-02,  7.49806244e-01,
        -1.80606900e-01,  2.97300933e-01,  2.28796057e-01,
         1.24925812e-01, -1.09812155e-02, -7.56400175e-01,
         3.06975327e-01, -1.46988798e-01,  6.39018554e-02,
        -2.27562870e-01,  1.25760255e-01,  3.75921941e-02,
        -9.44146804e-02, -4.98461260e-01, -1.77873941e-01,
        -5.07139893e-02, -2.48159959e-01,  5.67121026e-02,
        -1.69352948e-01,  9.14793269e-03, -5.15403130e-02,
        -4.39794991e-02,  1.04861286e-01, -1.10966296e-01,
         2.91294904e-02,  1.92105143e-01, -9.35986396e-02,
        -7.35301680e-01, -4.56775578e-01,  1.10240661e-01,
        -1.89737500e-02, -1.68843746e-01,  3.35458147e-01,
         2.22143828e-01,  2.83971107e-02,  5.07753854e-02,
         3.54751727e-01,  1.13864706e-01, -4.15141551e-01,
         3.88989129e-02, -1.27621604e-01,  2.60625584e-01,
         1.65599667e-01,  3.65434471e-01, -1.08361458e-01,
        -6.91749746e-01, -1.93517888e-01, -6.22065177e-02,
         3.86932476e-02, -6.31532753e-02, -1.09912198e-01,
         6.90058022e-02,  6.16434815e-01, -1.42034259e-01,
        -9.36271234e-02,  2.59216631e-01,  2.65891058e-01,
         6.88419013e-02, -2.38982442e-01, -1.79029213e-03,
         4.97254790e-01, -4.93357654e-02,  3.02103008e-02,
        -1.71738164e-01,  1.94155497e-03, -6.16430888e-01,
         2.51443013e-01, -3.75516008e-01, -2.30309321e-01,
         5.41654635e-02, -1.19916501e-01,  1.67007375e-01,
         6.81560336e-02, -3.21385893e-01,  2.89242920e-02,
        -5.05530588e-02, -1.95637019e-01,  1.04399753e-01,
        -3.90142842e-01, -2.71922264e-01, -2.57159765e-03,
        -6.40417544e-02,  4.33668716e-02,  8.40628958e-02,
        -1.23979548e-01,  3.95311486e-02, -2.46222454e-02,
        -2.10662393e-01, -2.70721519e-01, -1.55810684e-01,
         2.83889553e-02, -2.43731722e-01, -1.69693378e-03,
         1.55682249e-01,  2.17762979e-01, -1.36581062e-02,
        -3.27727288e-02, -5.49207644e-02, -1.01827252e-01,
         1.10663527e+00, -1.10052745e-01, -1.90769763e-01,
        -1.63184992e-01,  8.87936998e-02, -2.91891235e-02,
         6.65077213e-02, -4.89864488e-03,  5.31280190e-01,
        -7.16293159e-03, -2.35451575e-02,  1.36367045e-01,
         5.42175647e-01,  4.43187578e-01, -1.77284167e-01,
         4.52740621e-02, -2.25372791e-01,  8.18483870e-01,
        -3.12592562e-02,  4.89441227e-02,  2.28144369e-02,
        -5.06010933e-04, -2.36641130e-02, -1.78880889e-02,
        -5.52616809e-02, -2.60431955e-02,  8.66534516e-02,
        -1.94689457e-01,  2.56357284e-02, -8.64758838e-02,
         8.84159568e-02,  2.51772731e-01,  4.68293712e-02,
        -1.22346392e+00,  4.07166889e-01,  4.18455106e-02,
        -6.05211263e-02, -6.48267814e-01, -1.77798338e-01,
        -3.47284571e-02, -1.26608226e-01,  7.39596707e-02,
         7.57781712e-02,  1.39429356e-02, -3.66649266e-02,
         5.65140991e-02, -7.61599129e-02, -7.58919311e-03,
         1.06116572e+00,  2.28834859e-01, -5.15590095e-02,
        -7.98199963e-03,  8.69001986e-03,  1.95752795e-02,
        -1.42971273e-01,  1.41314201e-01,  2.20710935e-02,
        -7.27664514e-02, -4.85346998e-02,  6.17219516e-01,
        -7.16505463e-02, -5.06601531e-03, -2.15005183e-01,
        -1.99503701e-04,  1.74682052e-01, -9.46242985e-01,
        -1.37603767e-01,  8.09387167e-03, -8.16596969e-02,
         1.23267263e-02, -2.34144894e-01,  9.89061851e-02,
         1.36718162e+00,  4.38385207e-03,  8.13583862e-02,
         6.46153719e-02, -6.36450441e-02,  4.50658633e-01,
        -1.10666001e-02,  2.66149866e-01, -2.28663822e-02,
        -1.84044119e-02,  6.70407017e-01,  1.95938357e-01,
         1.96217321e-01,  1.72032533e-02,  5.31836923e-02,
         8.29248091e-01, -1.66437402e-01, -1.66464377e-02,
        -2.30759772e-02,  1.38818592e-01, -1.61173162e-01,
         5.92304742e-03, -9.33915088e-02,  2.71090452e-01,
        -8.11607331e-02, -4.78261305e-02, -3.74005635e-01,
         4.85396947e-02, -7.93035027e-02, -1.43652387e-01,
        -4.42548888e-01, -1.42986688e-01,  1.51175151e-01,
         7.45714494e-02, -7.04978975e-01,  1.89432257e-01,
        -1.06016700e-02,  2.88819123e-01,  4.04092135e-01,
         2.63285190e-01,  1.32203957e-03,  1.17015731e-01,
        -5.93588586e-02, -6.81358080e-03,  4.67243306e-03,
         2.32501266e-01,  1.69302336e-01, -8.18557692e-01,
        -3.36418197e-01,  1.30384731e-02,  1.64418820e-01,
         3.12525666e-02]])

centroid_animado = np.random.rand(1,512)
centroid_animado.shape

(1, 512)

In [6]:
# style_wav = torch.tensor([centroid_animado]).type(torch.long).cuda()
style_wav = np.array([centroid_animado])
style_wav.shape

(1, 1, 512)

In [18]:
centroid_animado

array([[0.62868911, 0.93010892, 0.65757366, 0.92075759, 0.94679794,
        0.99286925, 0.02644167, 0.66083707, 0.66520532, 0.26983003,
        0.44262309, 0.45914167, 0.84071672, 0.57852945, 0.98883635,
        0.69376909, 0.44216987, 0.48879031, 0.71979463, 0.2850419 ,
        0.42149138, 0.43011858, 0.49370274, 0.44757454, 0.47305246,
        0.90372368, 0.57404406, 0.58129445, 0.56153032, 0.59291029,
        0.89951747, 0.87262366, 0.46178924, 0.23182583, 0.39246522,
        0.59514444, 0.20269297, 0.24376578, 0.2960913 , 0.67755726,
        0.29495713, 0.43700719, 0.04443534, 0.34675929, 0.49662781,
        0.42369385, 0.51198131, 0.03215686, 0.39549595, 0.81383569,
        0.54278773, 0.67052818, 0.4238799 , 0.29807858, 0.95765057,
        0.13565372, 0.70394349, 0.73573806, 0.64534101, 0.40951303,
        0.85176877, 0.65724803, 0.71624639, 0.29519874, 0.04281235,
        0.53034208, 0.84948607, 0.68629733, 0.15414079, 0.8583511 ,
        0.2746398 , 0.43799747, 0.79755369, 0.46

In [8]:
from TTS.tts.utils.synthesis import synthesis
# style_wav = {"3": 0.1}
# style_wav = {"10": 0.1}
# style_wav = {"5": 0.05} # Esse token parece colocar uma voz mais grossa
# style_wav = {"8": 0.1} # Muda a tonalidade da voz feminina
# style_wav = {"6": 0.2} # Fica abafado e nao?
# style_wav = {'11': -0.05} # Uma voz meio misturada entre rosana e alfredo?
# style_wav = {'2': 0.1}
# style_wav = {'13': -0.07} # Parece um token interessante
# style_wav = {'0': 1.0}
# style_wav = {'0': 1.0}
# style_wav = {'1': 0.0}

# style_wav = {"0": 1, "1": 0}

# style_wav = torch.tensor([centroid_animado]).type(torch.long).cuda()

# f = 0
# style_wav = {}
# n_tokens = 2
# for i in range(n_tokens):
#   style_wav[str(i)] = f

# print(style_wav)

# style_wav = '/content/drive/MyDrive/Mestrado/TTS/audios_results/valid_30_gen.wav'
# style_wav = '/content/drive/MyDrive/Mestrado/TTS/audios_results/vctk_p247_126_gen.wav'
# style_wav = '/content/drive/MyDrive/Mestrado/TTS/GST_3speaker_CPQD/Animado7_16.wav'
# style_wav = '/content/drive/MyDrive/Mestrado/TTS/GST_3speaker_CPQD/Acolhedor7_16.wav'
# style_wav = '/content/drive/MyDrive/Mestrado/TTS/GST_3speaker_CPQD/expressivo_animado0178.wav'

speaker_embedding = None
speaker_id = 2

wav, alignment, decoder_output, postnet_output, stop_tokens, inputs, logits = synthesis(
    model,
    s,
    CONFIG,
    use_cuda,
    ap,
    speaker_id=speaker_id,
    speaker_embedding=speaker_embedding,
    style_wav=style_wav,
    truncated=False,
    enable_eos_bos_chars=CONFIG.enable_eos_bos_chars, #pylint: disable=unused-argument
    use_griffin_lim=True,
    do_trim_silence=False)

tensor([[[ 0.0214,  0.0118,  0.0037,  ...,  0.0052, -0.0046,  0.0007],
         [ 0.0184,  0.0207,  0.0092,  ...,  0.0004, -0.0111,  0.0069],
         [ 0.0450,  0.0304,  0.0198,  ..., -0.0072, -0.0164,  0.0085],
         ...,
         [ 0.0399,  0.0360,  0.0279,  ..., -0.0308, -0.0154, -0.0027],
         [ 0.0331,  0.0332,  0.0270,  ..., -0.0226, -0.0314,  0.0043],
         [ 0.0244,  0.0364,  0.0229,  ..., -0.0171, -0.0102,  0.0041]]],
       device='cuda:0') tensor([[[0.6287, 0.9301, 0.6576, 0.9208, 0.9468, 0.9929, 0.0264, 0.6608,
          0.6652, 0.2698, 0.4426, 0.4591, 0.8407, 0.5785, 0.9888, 0.6938,
          0.4422, 0.4888, 0.7198, 0.2850, 0.4215, 0.4301, 0.4937, 0.4476,
          0.4731, 0.9037, 0.5740, 0.5813, 0.5615, 0.5929, 0.8995, 0.8726,
          0.4618, 0.2318, 0.3925, 0.5951, 0.2027, 0.2438, 0.2961, 0.6776,
          0.2950, 0.4370, 0.0444, 0.3468, 0.4966, 0.4237, 0.5120, 0.0322,
          0.3955, 0.8138, 0.5428, 0.6705, 0.4239, 0.2981, 0.9577, 0.1357,
          0.7039

   | > Decoder stopped with 'max_decoder_steps


In [9]:
wav

array([ 0.408408 , -1.6478646, -2.3865576, ..., 10.093371 , 14.23977  ,
       18.295317 ], dtype=float32)

In [10]:
logits

tensor([[[-0.0577, -0.5627]]], device='cuda:0')

In [20]:
torch.tensor([centroid_animado]).type(torch.float).cuda()

tensor([[[0.6287, 0.9301, 0.6576, 0.9208, 0.9468, 0.9929, 0.0264, 0.6608,
          0.6652, 0.2698, 0.4426, 0.4591, 0.8407, 0.5785, 0.9888, 0.6938,
          0.4422, 0.4888, 0.7198, 0.2850, 0.4215, 0.4301, 0.4937, 0.4476,
          0.4731, 0.9037, 0.5740, 0.5813, 0.5615, 0.5929, 0.8995, 0.8726,
          0.4618, 0.2318, 0.3925, 0.5951, 0.2027, 0.2438, 0.2961, 0.6776,
          0.2950, 0.4370, 0.0444, 0.3468, 0.4966, 0.4237, 0.5120, 0.0322,
          0.3955, 0.8138, 0.5428, 0.6705, 0.4239, 0.2981, 0.9577, 0.1357,
          0.7039, 0.7357, 0.6453, 0.4095, 0.8518, 0.6572, 0.7162, 0.2952,
          0.0428, 0.5303, 0.8495, 0.6863, 0.1541, 0.8584, 0.2746, 0.4380,
          0.7976, 0.4647, 0.6147, 0.6250, 0.4688, 0.5047, 0.4157, 0.1241,
          0.3568, 0.5949, 0.8295, 0.8047, 0.6156, 0.9685, 0.9190, 0.0486,
          0.1204, 0.2992, 0.0578, 0.7202, 0.5595, 0.5383, 0.0345, 0.0237,
          0.4074, 0.7186, 0.8936, 0.8443, 0.0462, 0.2818, 0.6407, 0.4439,
          0.9470, 0.8432, 0.8607, 0.19

In [21]:
style_wav = torch.tensor([centroid_animado]).type(torch.float).cuda()
inputs, gst_outputs, logits = model.compute_gst(chars_var.unsqueeze(0), style_wav, None)

tensor([[[ 60,  47,  61, 101,  62,  61,  50,  51, 101,  54,  47,  68,  51,  64,
          101,  66,  64,  51,  53,  67,  47,  65, 101,  60,  47, 101,  53,  67,
           51,  64,  64,  47, 101,  49,  61,  60,  66,  64,  47, 101,  47, 101,
           59,  47,  58,  47,  64,  55,  47, 101,  97]]], device='cuda:0',
       dtype=torch.int32) tensor([[[0.6287, 0.9301, 0.6576, 0.9208, 0.9468, 0.9929, 0.0264, 0.6608,
          0.6652, 0.2698, 0.4426, 0.4591, 0.8407, 0.5785, 0.9888, 0.6938,
          0.4422, 0.4888, 0.7198, 0.2850, 0.4215, 0.4301, 0.4937, 0.4476,
          0.4731, 0.9037, 0.5740, 0.5813, 0.5615, 0.5929, 0.8995, 0.8726,
          0.4618, 0.2318, 0.3925, 0.5951, 0.2027, 0.2438, 0.2961, 0.6776,
          0.2950, 0.4370, 0.0444, 0.3468, 0.4966, 0.4237, 0.5120, 0.0322,
          0.3955, 0.8138, 0.5428, 0.6705, 0.4239, 0.2981, 0.9577, 0.1357,
          0.7039, 0.7357, 0.6453, 0.4095, 0.8518, 0.6572, 0.7162, 0.2952,
          0.0428, 0.5303, 0.8495, 0.6863, 0.1541, 0.8584, 0.2746, 0

In [22]:
chars_var.shape

torch.Size([1, 51])

In [23]:
gst_outputs.shape

torch.Size([1, 1, 512])

In [24]:
inputs

tensor([[[6.0000e+01, 4.7000e+01, 6.1000e+01, 1.0100e+02, 6.2000e+01,
          6.1000e+01, 5.0000e+01, 5.1000e+01, 1.0100e+02, 5.4000e+01,
          4.7000e+01, 6.8000e+01, 5.1000e+01, 6.4000e+01, 1.0100e+02,
          6.6000e+01, 6.4000e+01, 5.1000e+01, 5.3000e+01, 6.7000e+01,
          4.7000e+01, 6.5000e+01, 1.0100e+02, 6.0000e+01, 4.7000e+01,
          1.0100e+02, 5.3000e+01, 6.7000e+01, 5.1000e+01, 6.4000e+01,
          6.4000e+01, 4.7000e+01, 1.0100e+02, 4.9000e+01, 6.1000e+01,
          6.0000e+01, 6.6000e+01, 6.4000e+01, 4.7000e+01, 1.0100e+02,
          4.7000e+01, 1.0100e+02, 5.9000e+01, 4.7000e+01, 5.8000e+01,
          4.7000e+01, 6.4000e+01, 5.5000e+01, 4.7000e+01, 1.0100e+02,
          9.7000e+01, 6.2869e-01, 9.3011e-01, 6.5757e-01, 9.2076e-01,
          9.4680e-01, 9.9287e-01, 2.6442e-02, 6.6084e-01, 6.6521e-01,
          2.6983e-01, 4.4262e-01, 4.5914e-01, 8.4072e-01, 5.7853e-01,
          9.8884e-01, 6.9377e-01, 4.4217e-01, 4.8879e-01, 7.1979e-01,
          2.8504e-01

In [25]:
inputs.shape

torch.Size([1, 1, 563])

In [26]:
style_wav is None

False